# Задание 1
В файле содержится информация о покупках людей:

* **id** – означает покупку (в одну покупку входят все товары, купленные пользователем во время 1 похода в магазин)
* **Товар** – наименование товара
* **Количество** – число единиц купленного товара

Воспользуйтесь этими данными и выясните, какие пары товаров пользователи чаще всего покупают вместе.
Напишите код на **python** для получения нужной таблицы и укажите **5** наиболее распространённых паттернов.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Открытие csv-файла и проверка датафрейма

In [2]:
df = pd.read_csv('test1_completed.csv')

In [3]:
df.head()

,id,Товар,Количество
0,17119,Лимон,1.1
1,17119,Лимон оранжевый,0.7
2,17119,Лук-порей,10.0
3,17119,Лук репчатый,2.5
4,17119,Малина свежая,1.0


Проверяем датафрейм на наличие повторов и отсутствующих значений:

In [4]:
df.duplicated().sum()

0

In [5]:
df.isna().sum()

id            0
Товар         0
Количество    0
dtype: int64

Смотрим на минимальное **'Количество'** приобретаемых товаров:

In [6]:
df['Количество'].min()

0.1

Для определения паттернов встречаемости, количество товара в чеке неважно. Примем '**Количество**' = 1, для определения наличия товара в чеке:

In [7]:
df['Количество'] = 1

Для определения наиболее популярных пар товаров в чеке, удобно использовать сводные таблицы.

### Создание Pivot table и определение паттерна покупок.

Создадим сводную таблицу, в которой **index =** 'id чека', **columns =** 'Товар', **values =** 'встречаемость товара в чеке' и заполним пустые строки **NaN** нулями:

In [8]:
pivot_1 = df.pivot_table(index='Товар', columns='id', values='Количество', fill_value=0)

Руководствуюясь свойствами матриц, определим наиболее популярные пары товаров. Для этого создадим транспонированную матрицу **pivot_2**, а затем перемножим получившиеся сводные таблицы:

In [9]:
pivot_2=pivot_1.T

In [10]:
pair_pivot=pivot_1.dot(pivot_2)

In [11]:
pair_pivot

Товар,Абрикос вяленый,Абрикосы молдавские,Авокадо ХАСС,Авокадо стандарт,Алыча вяленая,Ананас Gold,Ананасовые кольца,Апельсины столовые,Арбуз,Арбуз овальный,...,Яблоки Гала,Яблоки Голден,Яблоки Джонаголд,Яблоки Мутсу,Яблоки Симиренко,Яблоки Фуджи,Яблоки Чемпион,Яблоки сезонные,Яблоки сушеные,Ягоды Годжи
Товар,,,,,,,,,,,,,,,,,,,,,
Абрикос вяленый,111,20,10,14,6,5,7,14,42,7,...,12,4,9,3,9,5,5,5,7,7
Абрикосы молдавские,20,471,37,46,23,23,19,48,141,37,...,30,26,29,23,35,26,27,23,18,29
Авокадо ХАСС,10,37,227,25,7,15,18,32,80,16,...,14,16,15,14,19,19,17,13,14,16
Авокадо стандарт,14,46,25,238,14,18,15,31,86,14,...,16,20,11,18,16,17,18,15,9,18
Алыча вяленая,6,23,7,14,142,9,9,23,41,9,...,9,6,11,10,5,9,9,9,13,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Яблоки Фуджи,5,26,19,17,9,6,9,15,48,14,...,11,11,13,13,22,156,10,5,10,8
Яблоки Чемпион,5,27,17,18,9,9,9,18,41,14,...,13,11,11,7,11,10,144,10,7,11
Яблоки сезонные,5,23,13,15,9,10,12,22,52,10,...,16,8,3,7,10,5,10,144,8,11


Мы получили частоту встречаемости каждой пары товаров в чеке, теперь решим проблему дублирования результатов относительно диагонали.

###  Фильтрация паразитных значений

Создадим маску для ненужных нам значений. Для этого воспользуемся методом **triu()** библиотеки **numpy**, он позволит преобразовать нижний треугольник относительно диагонали в **0**, затем обернем значения в тип данных **'bool'**:

In [12]:
value_mask = np.triu(pair_pivot.to_numpy()).astype(bool)

Теперь воспользуемся методом **mask** библиотеки **pandas** и наложим маску, полученную в предыдущем шаге, на значения **pair_pivot**, после чего заполним пустые значения нулями:

In [13]:
final_pairs = pair_pivot.mask(value_mask).fillna(0)

In [14]:
final_pairs

Товар,Абрикос вяленый,Абрикосы молдавские,Авокадо ХАСС,Авокадо стандарт,Алыча вяленая,Ананас Gold,Ананасовые кольца,Апельсины столовые,Арбуз,Арбуз овальный,...,Яблоки Гала,Яблоки Голден,Яблоки Джонаголд,Яблоки Мутсу,Яблоки Симиренко,Яблоки Фуджи,Яблоки Чемпион,Яблоки сезонные,Яблоки сушеные,Ягоды Годжи
Товар,,,,,,,,,,,,,,,,,,,,,
Абрикос вяленый,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Абрикосы молдавские,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Авокадо ХАСС,10.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Авокадо стандарт,14.0,46.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Алыча вяленая,6.0,23.0,7.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Яблоки Фуджи,5.0,26.0,19.0,17.0,9.0,6.0,9.0,15.0,48.0,14.0,...,11.0,11.0,13.0,13.0,22.0,0.0,0.0,0.0,0.0,0.0
Яблоки Чемпион,5.0,27.0,17.0,18.0,9.0,9.0,9.0,18.0,41.0,14.0,...,13.0,11.0,11.0,7.0,11.0,10.0,0.0,0.0,0.0,0.0
Яблоки сезонные,5.0,23.0,13.0,15.0,9.0,10.0,12.0,22.0,52.0,10.0,...,16.0,8.0,3.0,7.0,10.0,5.0,10.0,0.0,0.0,0.0


### Приведение сводной таблицы к нормальному виду

In [15]:
pairs = final_pairs.stack().to_frame()\
    .reset_index(1)\
    .rename(columns={'Товар':'Second'})\
    .reset_index()\
    .rename(columns={'Товар':'First', 0:'Buying_frequency'})\
    .sort_values('Buying_frequency', ascending=False)

### Проверка правильности работы алгоритма

Создадим запрос для двух конкретных товаров и посмотрим на частоту их встречаемости:

In [16]:
pairs.query("First == 'Яблоки Фуджи' and Second == 'Арбуз овальный'")

,First,Second,Buying_frequency
38615,Яблоки Фуджи,Арбуз овальный,14.0


В качестве контрольного значения используем данные полученные простым преобразованием исходного датафрейма, для этого сгруппируем его по чекам и товары в каждом чеке перечислим в отдельном списке:

In [17]:
test = df.groupby('id').agg({'Товар': lambda x: x.to_list()})
test['Товар'] = test['Товар'].astype(str)

Оставим только те чеки, которые содержат необходимую нам пару товаров и подсчитаем их количество:

 *так как мы используем метод **contains** необходимо использовать только точные названия товаров для проверки. Так **contains('Арбуз овальный')** в нашем случае, оставит лишь нужные значения, а **contains('Арбуз')** даст ложные значения, относящиеся также к другому товару*):

In [18]:
number_1 = test.loc[test['Товар'].str.contains('Яблоки Фуджи')]['Товар'].str.contains('Арбуз овальный').sum()

In [19]:
number_1

14

Количество совпадений в итоговой таблице совпадает с количеством совпадений в исходных данных. 

**Алгоритм работает правильно**.

Выводим 5 наиболее распространенных паттернов:

In [20]:
pairs.head()

,First,Second,Buying_frequency
32948,Укроп,Огурцы Луховицкие,431.0
32962,Укроп,Петрушка,408.0
22495,Огурцы Луховицкие,Арбуз,345.0
22542,Огурцы Луховицкие,Кабачки,326.0
32902,Укроп,Кинза,303.0
